In [20]:
import os
import sys
import datetime
import torch
import pandas as pd
import numpy as np
import time
import copy
import seaborn as sn
import matplotlib.pyplot as plt

from pytorch_lightning.metrics.classification import F1
from tqdm import tqdm

In [3]:
from IPython.display import clear_output

In [5]:
data_root = "../dataset"
images_root = os.path.join(data_root, "images_all")
masks_root = os.path.join(data_root, "masks_all")

In [6]:
test_data_dist = os.path.join(data_root, "test_data.csv")

In [7]:
scripts_path = "../scripts"

In [8]:
sys.path.append(scripts_path)

In [9]:
import constants as const

from data_loader import MelanomaClassificationDataset, MelanomaSegmentationDataset
from seg_train_utils import get_data_loader

In [10]:
test_data = pd.read_csv(test_data_dist)

In [11]:
test_data.head()

,name,class
0,ISIC_0005590.png,benign
1,ISIC_0006690.png,benign
2,ISIC_0010705.png,benign
3,ISIC_0011791.png,benign
4,ISIC_0000428.png,malignant


In [12]:
test_data_tr = test_data.copy()
test_data_tr = test_data_tr.replace({"class": {"benign": 0, "malignant": 1}})

In [13]:
test_data_tr.head()

,name,class
0,ISIC_0005590.png,0
1,ISIC_0006690.png,0
2,ISIC_0010705.png,0
3,ISIC_0011791.png,0
4,ISIC_0000428.png,1


In [14]:
print("We have {} benign data points".format(len(test_data_tr[test_data_tr["class"] == 0])))
print("We have {} malignant data points".format(len(test_data_tr[test_data_tr["class"] == 1])))

We have 1159 benign data points
We have 226 malignant data points


In [15]:
test_dataset_classification = MelanomaClassificationDataset(csv_file = test_data_tr, 
                                             root_dir = images_root,
                                             augmentation = None,
                                             preprocessing = MelanomaClassificationDataset.get_default_preprocessing())


test_dataset_segmentation = MelanomaSegmentationDataset(csv_file = test_data_tr,
                                                        root_dir = (images_root, masks_root),
                                                        augmentation = None,
                                                        preprocessing = MelanomaSegmentationDataset.get_default_preprocessing())

In [17]:
test_loader_classificaiton = get_data_loader(test_dataset_classification, batch_size = const.batch_size_val, shuffle=False, num_workers = 0)

test_loader_segmentation = get_data_loader(test_dataset_segmentation, batch_size = const.batch_size_val, shuffle=False, num_workers = 0)

In [18]:
model_classification = torch.load("../models/classification_model_inception_v3.279314.pth")
model_classification.eval()

model_segmentation = torch.load("../models/segmentation_model_xception_backbone.pth")
model_segmentation.eval()

FPN(
  (encoder): XceptionEncoder(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [19]:
res = pd.DataFrame(columns = ["prediction", "ground_truth"])

with torch.no_grad(): 
    for image, label in tqdm(test_loader):
        image = image.to(device)
        label = label.to(device)

        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        
        res = res.append({
            "prediction": preds.item(), 
            "ground_truth": label.item()
        }, ignore_index = True)
        
    preds_all = torch.tensor(res.prediction.values.astype(int))
    gt_all = torch.tensor(res.ground_truth.values.astype(int))

    conf_matrix = confusion_matrix(gt_all, preds_all)

    print("Precision: {:.2f}".format(precision_score(gt_all, preds_all)))
    print("Recall: {:.2f}".format(recall_score(gt_all, preds_all)))
    print("Accuracy: {:.2f}".format(accuracy_score(gt_all, preds_all)))
    print("F1 score: {:.2f}".format(f1_score(gt_all, preds_all)))
    print("Confusion matrix:\n{}".format(conf_matrix))

100%|██████████| 1385/1385 [01:34<00:00, 14.60it/s]

Precision: 0.42
Recall: 0.74
Accuracy: 0.79
F1 score: 0.53
Confusion matrix:
[[924 235]
 [ 59 167]]
